# Hayden's Notebook

I've reduced the number of imports here to those you might actually need. You will need either the NonnegativeMatrixFactorization or the LatentDiricheletAllocation for topic modeling. (Go back to those notebooks for more.)

In [1]:
import re
import numpy as np
import pandas as pd
from pathlib import Path
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
films = []
for p in Path("../queue/Mystery").glob('*.txt'):
    with open(p, mode="r", encoding="utf-8") as f:
        film = f.read()
        films.append(film)

Instead of two separate loops through the films, one to tokenize and one to break the tokens into three groups, I am combining those two tasks into your **`film_chunker`** function. It's completely fine to do it in two steps. I am reducing it to one for the ease of readability (for me).

In [3]:
def film_chunker(film):
    ''' takes a filmscript as a string and tokenizes it and
        returns three lists of tokens as approximate stand-ins
        for Acts I, II, and III in the Hollywood formula
    '''
    # Tokenize the string
    tokenized_film = word_tokenize(film)

    # Break the list into three lists:
    ## Set our chunk sizes
    chunk1_size = 0.25
    chunk2_size = 0.50
    chunk3_size = 0.25

    # Calculate the sizes of each chunk based on the length of the file
    total_length = len(tokenized_film)
    chunk1_end = int(total_length * chunk1_size)
    chunk2_end = int(total_length * (chunk1_size + chunk2_size))

    # Divide the file into three chunks
    chunk1 = tokenized_film[:chunk1_end]
    chunk2 = tokenized_film[chunk1_end:chunk2_end]
    chunk3 = tokenized_film[chunk2_end:]

    # Store the chunks for this film in a tuple
    chunks = (chunk1, chunk2, chunk3)
    
    return chunks

The script above now handles only one string (film) at a time and returns a tuple consisting of three lists of tokenized words. E.g.:
```
chunks = ( ['word', 'word', 'word'], ['word', 'word', 'word'], ['word', 'word'] )
```
What the code below does is run each film through the film chunker and return the tuple, and, since it's running through a list of strings, it returns a list of tuples. 
```
chunked_films = [ ([], [], []), ([], [], []), ([], [], []), ... ]
```
What's below is a list comprehension, which is simply a more compact form of a for loop that doesn't require you to create an empty list first and then append it. In other words, you could do the same thing with the following:
```python
chunked_films =[]
for film in films:
    chunks = film_chunker(film)
    chunked_films.append(chunks)
```

In [4]:
%%time
chunked_films = [ film_chunker(film) for film in films ]

CPU times: user 18.2 s, sys: 63.5 ms, total: 18.2 s
Wall time: 18.3 s


To see the nested structure, you can keep slicing -- which also reveals how you can access each level.

In [5]:
# What's the top level structure
print(type(chunked_films))

# Okay, it's a list of:
print(type(chunked_films[0]))

# And each tuple is made up of:
print(type(chunked_films[0][0]))

# And the bottom turtle is:
print(type(chunked_films[0][0][0]))

<class 'list'>
<class 'tuple'>
<class 'list'>
<class 'str'>


I think the simplest way to re-package things is with a `for` loop. Some might argue that it's not very compact, but it gets the job done.

In [6]:
%%time
act1s = []
act2s = []
act3s = []
for item in chunked_films:
    act1s.append(item[0])
    act2s.append(item[1])
    act3s.append(item[2])

CPU times: user 88 µs, sys: 162 µs, total: 250 µs
Wall time: 251 µs


In [7]:
print(f"The first film in our list is {films[0][20:45]}")
print(f"It's {len(word_tokenize(films[0]))} words long.")
print(f"The first act is {len(act1s[0])}.")

The first film in our list is         RED RIDING HOOD


It's 34214 words long.
The first act is 8553.


That number looks to be about a quarter of the overall length, so we appear to be in business!

Now, treat `act1s` as a collection of documents and topic model them. The same goes for `act2s` and `act3s`. (You may need to join the lists into strings in order to feed them into the CountVectorizer, if you use LDA, or the TfidfVectorizer if you use NMF.